In [1]:
import numpy as np
from time import perf_counter, sleep
import matplotlib.pyplot as plt
from tqdm import tqdm

from board import board_obj
from operations import ops
import vis_tools
from IPython.display import clear_output
import time
import copy
import math
import scipy.stats as stats

In [2]:
def faceoff_sequential(agent1, agent2, ngames=100, visualize=False):
    ''' SLOW BUT RELIABLE, TO BE USED WHEN THE RESULTS FROM FACEOFF_PARALLEL ARE SUSPICIOUS '''

    win_counter = 0 # quick integer check to see whether line bot wins more than loses
    loss_counter = 0
    draw_counter = 0

    for j in tqdm(range(ngames)):
        my_board = board_obj()
        #make first 4 moves randomly
        for i in range(4):
            legal_moves = ops.get_valid_moves(my_board)
            move = legal_moves[np.random.choice(len(legal_moves))]
            ops.make_move(my_board, move)

        for i in range(81): # up to 81 moves per game.
            ''' ------ agent 1 turn ------'''
            # get dictionary 
            temp_dict = ops.pull_dictionary(my_board)
            # give dict to agent, calculate move
            start = perf_counter()
            agent1_move = agent1.move(temp_dict)
            # time.sleep(0.5)
            # validate the move
            if not ops.check_move_is_valid(my_board, agent1_move):
                raise Exception(f'invalid move selected by p1, {agent1_move}')

            # make the move
            ops.make_move(my_board, agent1_move)
            if visualize:
                plt.clf()
                clear_output(wait=True)
                vis_tools.fancy_draw_board(my_board)
                plt.show()
            # check whether game is finished
            if ops.check_game_finished(my_board):
                if 'agent 1' in ops.get_winner(my_board):
                    win_counter += 1
                else:
                    draw_counter += 1
                break

            ''' agent 2 turn '''
            # get dictionary 
            temp_dict = ops.pull_dictionary(my_board)
            # give dict to agent, calculate move
            start = perf_counter()
            agent2_move = agent2.move(temp_dict)
            # time.sleep(0.5)

            # validate the move
            if not ops.check_move_is_valid(my_board, agent2_move):
                raise Exception(f'invalid move selected by p2, {agent2_move}')
            # make the move
            ops.make_move(my_board, agent2_move)
            if visualize:
                plt.clf()
                clear_output(wait=True)
                vis_tools.fancy_draw_board(my_board)
                plt.show()
            # check whether game is finished
            if ops.check_game_finished(my_board):
                if 'agent 2' in ops.get_winner(my_board):
                    loss_counter += 1
                else:
                    draw_counter += 1
                break
            if visualize:
                
                print(win_counter, loss_counter, draw_counter)
    # Calculate Elo difference
    total_games = win_counter + loss_counter + draw_counter
    win_rate = win_counter / total_games
    draw_rate = draw_counter / total_games
    loss_rate = loss_counter / total_games
    E = win_rate + 0.5 * draw_rate
    elo_diff = -400 * math.log10(1 / E - 1)

    #ci formula from view-source:https://3dkingdoms.com/chess/elo.htm
    percentage = (win_counter + draw_counter / 2) / total_games
    
    wins_dev = win_rate * (1- percentage)**2
    draws_dev = draw_rate * (0.5 - percentage)**2
    losses_dev = loss_rate * (0 - percentage)**2

    std_dev = math.sqrt(wins_dev + draws_dev + losses_dev) / math.sqrt(total_games)

    confidence = 0.95
    min_confidence = (1- confidence) / 2
    max_confidence = 1 - min_confidence

    min_dev = percentage + stats.norm.ppf(min_confidence) * std_dev
    max_dev = percentage + stats.norm.ppf(max_confidence) * std_dev
    diff = (-400 * math.log10(1 / max_dev - 1)) - (-400 * math.log10(1 / min_dev - 1)) 
    
    return {'win':win_counter, 'loss':loss_counter, 'draw':draw_counter, 'elo_diff':elo_diff, 'elo_diff_ci +/': diff}

In [3]:
from joblib import Parallel, delayed

def play_single_game(agent1, agent2):
    win, draw, loss = 0, 0, 0
    my_board = board_obj()
    #make first 4 moves randomly
    for _ in range(4):
        legal_moves = ops.get_valid_moves(my_board)
        move = legal_moves[np.random.choice(len(legal_moves))]
        ops.make_move(my_board, move)

    for _ in range(81): # up to 81 moves per game.
        ''' ------ agent 1 turn ------'''
        # get dictionary 
        temp_dict = ops.pull_dictionary(my_board)
        # give dict to agent, calculate move
        agent1_move = agent1.move(temp_dict)
        # validate the move
        if not ops.check_move_is_valid(my_board, agent1_move):
            raise Exception(f'invalid move selected by p1, {agent1_move}')

        # make the move
        ops.make_move(my_board, agent1_move)
        # check whether game is finished
        if ops.check_game_finished(my_board):
            if 'agent 1' in ops.get_winner(my_board):
                win += 1
            else:
                draw += 1
            break

        ''' agent 2 turn '''
        # get dictionary 
        temp_dict = ops.pull_dictionary(my_board)
        # give dict to agent, calculate move
        start = perf_counter()
        agent2_move = agent2.move(temp_dict)

        # validate the move
        if not ops.check_move_is_valid(my_board, agent2_move):
            raise Exception(f'invalid move selected by p2, {agent2_move}')
        # make the move
        ops.make_move(my_board, agent2_move)
        # check whether game is finished
        if ops.check_game_finished(my_board):
            if 'agent 2' in ops.get_winner(my_board):
                loss += 1
            else:
                draw += 1
            break
    return win, loss, draw

def faceoff_parallel(agent1, agent2, ngames=100, njobs=6):
    '''FAST, BUT RESULTS WILL BE SLIGHTLY DIFFERENT FROM SEQUENTIAL VERSION DUE TO PROCESSES COMPETING FOR RESOURCES.'''
    # Parallel execution with tqdm progress update
    results = Parallel(n_jobs=njobs, backend="loky")(delayed(play_single_game)(agent1(), agent2()) for _ in tqdm(range(ngames)))

    # Aggregate results
    total_wins = sum(result[0] for result in results)
    total_losses = sum(result[1] for result in results)
    total_draws = sum(result[2] for result in results)

    # Calculate Elo difference
    total_games = total_wins + total_losses + total_draws
    win_rate = total_wins / total_games
    draw_rate = total_draws / total_games
    loss_rate = total_losses / total_games
    E = win_rate + 0.5 * draw_rate
    elo_diff = -400 * math.log10(1 / E - 1)

    #ci formula from view-source:https://3dkingdoms.com/chess/elo.htm
    percentage = (total_wins + total_draws / 2) / total_games
    
    wins_dev = win_rate * (1- percentage)**2
    draws_dev = draw_rate * (0.5 - percentage)**2
    losses_dev = loss_rate * (0 - percentage)**2

    std_dev = math.sqrt(wins_dev + draws_dev + losses_dev) / math.sqrt(total_games)

    confidence = 0.95
    min_confidence = (1- confidence) / 2
    max_confidence = 1 - min_confidence

    min_dev = percentage + stats.norm.ppf(min_confidence) * std_dev
    max_dev = percentage + stats.norm.ppf(max_confidence) * std_dev
    diff = (-400 * math.log10(1 / max_dev - 1)) - (-400 * math.log10(1 / min_dev - 1)) 
    


    return {'win': total_wins, 'loss': total_losses, 'draw': total_draws, 'elo_diff': elo_diff, 'elo_conf_interval +/-': diff/2}

In [4]:
class random_bot:
    '''
    this bot selects a random valid move
    '''
    def __init__(self, name = 'beep-boop'):
        self.name = name
    def move(self, board_dict):
        # print(board_dict['valid_moves'])
        b = board_obj()
        b.build_from_dict_gamestate(board_dict)
        # print(b.miniboxes)
        random_index = np.random.choice(len(board_dict['valid_moves']))
        return board_dict['valid_moves'][random_index]

In [5]:
import numpy as np
class line_completer_bot:
    '''
    tries to complete lines, otherwise it plays randomly
    designed to show how to implement a relatively simple strategy
    '''
    
    ''' ------------------ required function ---------------- '''
    
    def __init__(self,name: str = 'Chekhov') -> None:
        self.name = name
        self.box_probs = np.ones((3,3)) # edges
        self.box_probs[1,1] = 4 # center
        self.box_probs[0,0] = self.box_probs[0,2] = self.box_probs[2,0] = self.box_probs[2,2] = 2 # corners
        
    def move(self, board_dict: dict) -> tuple:
        ''' wrapper
        apply the logic and returns the desired move
        '''
        return tuple(self.heuristic_mini_to_major(board_state = board_dict['board_state'],
                                                  active_box = board_dict['active_box'],
                                                  valid_moves = board_dict['valid_moves']))
    
    
    ''' --------- generally useful bot functions ------------ '''
    
    def _check_line(self, box: np.array) -> bool:
        '''
        box is a (3,3) array
        returns True if a line is found, else returns False '''
        for i in range(3):
            if abs(sum(box[:,i])) == 3: return True # horizontal
            if abs(sum(box[i,:])) == 3: return True # vertical

        # diagonals
        if abs(box.trace()) == 3: return True
        if abs(np.rot90(box).trace()) == 3: return True
        return False

    def _check_line_playerwise(self, box: np.array, player: int = None):
        ''' returns true if the given player has a line in the box, else false
        if no player is given, it checks for whether any player has a line in the box'''
        if player == None:
            return self._check_line(box)
        if player == -1:
            box = box * -1
        box = np.clip(box,0,1)
        return self._check_line(box)
    
    def pull_mini_board(self, board_state: np.array, mini_board_index: tuple) -> np.array:
        ''' extracts a mini board from the 9x9 given the its index'''
        temp = board_state[mini_board_index[0]*3:(mini_board_index[0]+1)*3,
                           mini_board_index[1]*3:(mini_board_index[1]+1)*3]
        return temp

    def get_valid(self, mini_board: np.array) -> np.array:
        ''' gets valid moves in the miniboard'''
#        print(mini_board)
#        print(np.where(mini_board == 0))
#        return np.where(mini_board == 0)
        return np.where(abs(mini_board) != 1)

    def get_finished(self, board_state: np.array) -> np.array:
        ''' calculates the completed boxes'''
        self_boxes = np.zeros((3,3))
        opp_boxes = np.zeros((3,3))
        stale_boxes = np.zeros((3,3))
        # look at each miniboard separately
        for _r in range(3):
            for _c in range(3):
                player_finished = False
                mini_board = self.pull_mini_board(board_state, (_r,_c))
                if self._check_line_playerwise(mini_board, player = 1):
                    self_boxes[_r,_c] = 1
                    player_finished = True
                if self._check_line_playerwise(mini_board, player = -1):
                    opp_boxes[_r,_c] = 1
                    player_finished = True
                if (sum(abs(mini_board.flatten())) == 9) and not player_finished:
                    stale_boxes[_r,_c] = 1

        # return finished boxes (separated by their content)
        return (self_boxes, opp_boxes, stale_boxes)
    
    def complete_line(self, mini_board: np.array) -> list:
        if sum(abs(mini_board.flatten())) == 9:
            print('invalid mini_board') # should never reach here
        # works as expected, however mini-board sometimes is sometimes invalid
        ''' completes a line if available '''
        # loop through valid moves with hypothetic self position there.
        # if it makes a line it's an imminent win
        imminent = list()
        valid_moves = self.get_valid(mini_board)
        for _valid in zip(*valid_moves):
            # create temp valid pattern
            valid_filter = np.zeros((3,3))
            valid_filter[_valid[0],_valid[1]] = 1
            if self._check_line(mini_board + valid_filter):
                imminent.append(_valid)
        return imminent
    
    def get_probs(self, valid_moves: list) -> np.array:
        ''' match the probability with the valid moves to weight the random choice '''
        valid_moves = np.array(valid_moves)
        probs = list()
        for _valid in np.array(valid_moves).reshape(-1,2):
            
            probs.append(self.box_probs[_valid[0],_valid[1]])
        probs /= sum(probs) # normalize
        return probs
    
    ''' ------------------ bot specific logic ---------------- '''
    
    def heuristic_mini_to_major(self,
                                board_state: np.array,
                                active_box: tuple,
                                valid_moves: list) -> tuple:
        '''
        either applies the heuristic to the mini-board or selects a mini-board (then applies the heuristic to it)
        '''
        if active_box != (-1,-1):
            # look just at the mini board
            mini_board = self.pull_mini_board(board_state, active_box)
            # look using the logic, select a move
            move = self.mid_heuristic(mini_board)
            # project back to original board space
            return (move[0] + 3 * active_box[0],
                    move[1] + 3 * active_box[1])

        else:
        #    print(np.array(valid_moves).shape) # sometimes the miniboard i'm sent to has no valid moves
        
            # use heuristic on finished boxes to select which box to play in
            imposed_active_box = self.major_heuristic(board_state)
#            print(self.pull_mini_board(board_state, imposed_active_box),'\n')
#            print('\n')

            # call this function with the self-imposed active box
            return self.heuristic_mini_to_major(board_state = board_state,
                                                active_box = imposed_active_box,
                                                valid_moves = valid_moves)

    def major_heuristic(self, board_state: np.array) -> tuple:
        '''
        determines which miniboard to play on
        note: having stale boxes was causing issues where the logic wanted to block
              the opponent but that mini-board was already finished (it was stale)
        '''
        z = self.get_finished(board_state)
        # finished boxes is a tuple of 3 masks: self, opponent, stale 
        self_boxes  = z[0]
        opp_boxes   = z[1]
        stale_boxes = z[2]
#        print('self:\n',self_boxes)
#        print('opp :\n',opp_boxes)
#        print('stale:\n',stale_boxes)
        
        # ----- identify imminent wins -----
        imminent_wins = self.complete_line(self_boxes)
#        print('len imminent win:',len(imminent_wins))
        # remove imminent wins that point to stale boxes (or opponent)
        stale_boxes_idxs = zip(*np.where(stale_boxes))
        for stale_box in stale_boxes_idxs:
            if stale_box in imminent_wins:
                imminent_wins.remove(stale_box)
        opp_boxes_idx = zip(*np.where(opp_boxes))
        for opp_box in opp_boxes_idx:
            if opp_box in imminent_wins:
                imminent_wins.remove(opp_box)
        # if it can complete a line, do it
        if len(imminent_wins) > 0: 
#            print('returning line')
#            print('len imminent win:',len(imminent_wins))
            return imminent_wins[np.random.choice(len(imminent_wins), p=self.get_probs(imminent_wins))]

        # ------ attempt to block -----
        imminent_loss = self.complete_line(opp_boxes)
        # make new list to remove imminent wins that point to stale boxes
        stale_boxes_idx = zip(*np.where(stale_boxes))
        for stale_box in stale_boxes_idx:
            if stale_box in imminent_loss:
                imminent_loss.remove(stale_box)
        self_boxes_idx = zip(*np.where(self_boxes))
        for self_box in self_boxes_idx:
            if self_box in imminent_loss:
                imminent_loss.remove(self_box)
        if len(imminent_loss) > 0:
#            print('returning block')
            return imminent_loss[np.random.choice(len(imminent_loss), p=self.get_probs(imminent_loss))]

        # ------ else take random ------
#        print('returning random')
        internal_valid = np.array(list(zip(*self.get_valid(self_boxes + opp_boxes + stale_boxes))))
        return tuple(internal_valid[np.random.choice(len(internal_valid), p=self.get_probs(internal_valid))])
        
    def mid_heuristic(self, mini_board: np.array) -> tuple:
        ''' main mini-board logic '''
        # try to complete a line on this miniboard
        imminent_wins = self.complete_line(mini_board)
        if len(imminent_wins) > 0:
            return imminent_wins[np.random.choice(len(imminent_wins))]

        ''' attempt to block'''
        imminent_wins = self.complete_line(mini_board * -1) # pretend to make lines from opponent's perspective
        if len(imminent_wins) > 0:
            return imminent_wins[np.random.choice(len(imminent_wins))]

        # else play randomly
        valid_moves = np.array(list(zip(*self.get_valid(mini_board))))
        return tuple(valid_moves[np.random.choice(len(valid_moves), p=self.get_probs(valid_moves))])



In [6]:
faceoff_parallel(line_completer_bot, random_bot, ngames=1000)

100%|██████████| 1000/1000 [00:04<00:00, 216.51it/s]


{'win': 945,
 'loss': 8,
 'draw': 47,
 'elo_diff': 595.1156285062111,
 'elo_conf_interval +/-': 49.52153938985663}

In [7]:
import time
class minimax_ref:
    def __init__(self,name: str = 'Minimax Reference') -> None:
        self.name = name
        self.thinking_time = 0.1
        self.root_best_move = None
        self.start_time = None
        self.score = 0
        self.maximizing_idx = 0
    def move(self, board_dict: dict) -> tuple:
        ''' wrapper
        apply the logic and returns the desired move
        '''
        b_obj = board_obj()
        b_obj.build_from_dict_gamestate(board_dict)
        self.maximizing_idx = b_obj.n_moves % 2
        return self.get_best_move(b_obj)
    
    def get_best_move(self, board: board_obj):
        depth = 1
        self.start_time = time.time()
        while time.time() - self.start_time < self.thinking_time:
            self.search(board, depth, True, 0)
            depth += 1
        # print(f'reached depth {depth-1} in {time.time() - self.start_time} seconds with score {self.score}')
        return self.root_best_move
    
    def search(self, board:board_obj, depth:int, maximizing_player:bool, ply: int) -> int:
        '''simple minimax search'''
        if ops.check_game_finished(board):
            if np.all(np.any(board.miniboxes,axis=2)):
                return 0 #draw
            else:
                if maximizing_player:
                    return -100 + ply
                else:
                    return 100 - ply
        if depth == 0:
            return self.evaluate(board)
        if time.time() - self.start_time > self.thinking_time:
            #want to immediately return and ignore results when out of time, so just turn node into a cutoff for its parent
            if maximizing_player:
                return np.inf
            else:
                return -np.inf
        if maximizing_player:
            max_value = -np.inf
            legal_moves = ops.get_valid_moves(board)
            for move in legal_moves:
                ops.make_move(board, move)
                new_value = self.search(board, depth-1, False, ply+1)
                if new_value > max_value:
                    max_value = new_value
                    if ply == 0:
                        self.root_best_move = move
                        self.score = max_value
                ops.undo_move(board)
            return max_value
        else:
            value = np.inf
            legal_moves = ops.get_valid_moves(board)
            for move in legal_moves:
                ops.make_move(board, move)
                value = min(value, self.search(board, depth-1, True, ply+1))
                ops.undo_move(board)
            return value
    def evaluate(self, board):
        '''simple evaluation function'''
        return self.minibox_score(board)
    def minibox_score(self, board):
        scores = [np.sum(board.miniboxes[:, :, p]) for p in range(2)]
        minibox_scores = scores[self.maximizing_idx] - scores[(self.maximizing_idx + 1) % 2]
        return minibox_scores

        


In [8]:
faceoff_parallel(minimax_ref, random_bot, ngames=1000, njobs=-1)

100%|██████████| 1000/1000 [02:58<00:00,  5.61it/s]


{'win': 773,
 'loss': 169,
 'draw': 58,
 'elo_diff': 243.0036712090531,
 'elo_conf_interval +/-': 25.855746711016167}

In [9]:
faceoff_parallel(minimax_ref, line_completer_bot, ngames=1000, njobs=-1)

100%|██████████| 1000/1000 [02:12<00:00,  7.55it/s]


{'win': 103,
 'loss': 843,
 'draw': 54,
 'elo_diff': -330.2303601247127,
 'elo_conf_interval +/-': 30.244754001318967}

In [10]:
class ab_pruning_ref(minimax_ref):
    def search(self, board:board_obj, depth:int, maximizing_player:bool, ply: int, alpha: int = -np.inf, beta: int = np.inf) -> int:
        '''simple minimax search'''
        if ops.check_game_finished(board):
            if np.all(np.any(board.miniboxes,axis=2)):
                return 0 #draw
            else:
                if maximizing_player:
                    return -100 + ply
                else:
                    return 100 - ply
        if depth == 0:
            return self.evaluate(board)
        if time.time() - self.start_time > self.thinking_time:
            if maximizing_player:
                return np.inf
            else:
                return -np.inf
        if maximizing_player:
            max_value = -np.inf
            legal_moves = ops.get_valid_moves(board)
            for move in legal_moves:
                ops.make_move(board, move)
                max_value = max(max_value, self.search(board, depth-1, False, ply+1, alpha, beta))
                ops.undo_move(board)
                if max_value > beta:
                    break
                if max_value > alpha:
                    alpha = max_value
                    if ply == 0:
                        self.root_best_move = move
                        self.score = max_value
            return max_value
        else:
            value = np.inf
            legal_moves = ops.get_valid_moves(board)
            for move in legal_moves:
                ops.make_move(board, move)
                value = min(value, self.search(board, depth-1, True, ply+1, alpha, beta))
                ops.undo_move(board)
                if value < alpha:
                    break
                beta = min(beta, value)
            return value


In [11]:
faceoff_parallel(ab_pruning_ref, random_bot, ngames=1000, njobs=-1)

100%|██████████| 1000/1000 [03:05<00:00,  5.39it/s]


{'win': 789,
 'loss': 139,
 'draw': 72,
 'elo_diff': 269.3663599454524,
 'elo_conf_interval +/-': 26.629082627501077}

In [12]:
faceoff_parallel(ab_pruning_ref, minimax_ref, ngames=1000, njobs=-1)

100%|██████████| 1000/1000 [05:56<00:00,  2.81it/s]


{'win': 578,
 'loss': 328,
 'draw': 94,
 'elo_diff': 88.73949984654251,
 'elo_conf_interval +/-': 21.1265662597706}

In [13]:
faceoff_parallel(ab_pruning_ref, line_completer_bot, ngames=1000, njobs=-1)

100%|██████████| 1000/1000 [02:19<00:00,  7.18it/s]


{'win': 102,
 'loss': 836,
 'draw': 62,
 'elo_diff': -325.6669826036498,
 'elo_conf_interval +/-': 29.687141188092994}